In [ ]:
!apt-get install git

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git is already the newest version (1:2.25.1-1ubuntu3.11).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [ ]:
!git clone https://github.com/mikasso/nasdaq_forecasting

fatal: destination path '.' already exists and is not an empty directory.


In [ ]:
!pip install -r requirements.txt -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached darts-0.23.1-py3-none-any.whl (592 kB)
  Attempting uninstall: darts
    Found existing installation: darts 0.24.0
    Uninstalling darts-0.24.0:
      Successfully uninstalled darts-0.24.0


In [ ]:
import sys
import shutil
sys.path.append('/content/nasdaq_forecasting')


In [ ]:
import enum
import logging
import model_rnn
import model_tft
import model_transformer
import _predict
from const import ModelConfig, RNN_NETWORKS, ModelTypes
from utils import create_folder, assert_pytorch_is_using_gpu
import validate
import view_results
from datasets import SeqDataset, Datasets, DatasetAccesor, DatasetTransformer, load_datasets
from google.colab import drive
import shutil

logging.basicConfig(level=logging.INFO)
LOGGER = logging.getLogger(name="Main runner")


def dispatch_model_training(config: ModelConfig):
    if config.model_type in RNN_NETWORKS:
        model_rnn.main(config)
    elif config.model_type == ModelTypes.tft:
        model_tft.main(config)
    elif config.model_type == ModelTypes.transformer:
        model_transformer.main(config)
    else:
        LOGGER.warning(f"Unrecognized model type {config.model_type}")


if __name__ == "__main__":
    assert_pytorch_is_using_gpu()
    drive.mount('/content/drive')
    models = [
        ModelConfig(ModelTypes.rnn, 1),
        ModelConfig(ModelTypes.gru, 1),
        ModelConfig(ModelTypes.lstm, 1),
        ModelConfig(ModelTypes.transformer, 1),
        ModelConfig(ModelTypes.tft, 1),
        ModelConfig(ModelTypes.rnn, 7),
        ModelConfig(ModelTypes.gru, 7),
        ModelConfig(ModelTypes.lstm, 7),
        ModelConfig(ModelTypes.transformer, 7),
        ModelConfig(ModelTypes.tft, 7),
    ]

    for model_config in models:
        LOGGER.info(f"Running model: {model_config.model_name}")
        create_folder(model_config.result_path, delete_if_exists=True)
        dispatch_model_training(model_config)
        validate.main(model_config)
        LOGGER.info(f"Finished running model: {model_config.model_name}")
        shutil.copytree(f"results/{model_config.model_name}", f"drive/MyDrive/nasdaq/results/{model_config.model_name}", dirs_exist_ok=True)  
        shutil.copytree(f"darts_logs/{model_config.model_name}", f"drive/MyDrive/nasdaq/models/{model_config.model_name}", dirs_exist_ok=True)  
        break
 
  
   


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'drive/MyDrive/nasdaq/models'